# Clustering

## 1. keyword 정제
    1) 주요 태그 추출
    - description 내용 포함 태그
    - description과 일치하는 단어가 없는 태그는 태그 그대로 사용
    - 태그 없는 187개 스튜디오는 description에서 추출한 단어 사용
     
     2) 유사 단어 묶어주기
    - WordCloudVec으로 단어간 유사도 판별

    >> 타겟 스튜디오와 동일한 태그 개수 세기

## 2. 범주형 데이터 Gower로 수치형 데이터로 변환
    1) 범주형 데이터는 category와 

In [1]:
import numpy as np 
import pandas as pd 
from pandas import DataFrame as df
import re # 정규식 사용을 위함
import nltk
import csv
from konlpy.tag import Okt
from konlpy.utils import pprint
from collections import Counter

In [807]:
import os
dataset = pd.read_csv("total_dataset_trim.csv",encoding='utf-8')
dataset.tail()

,NO,img_fold_no,studio_name,studio_picture,imafe_file,category,region,tag,price,area,floor,default_person,parking,description
1847,1804,89,홍대 2룸,['https://s3.hourplace.co.kr/image/user/841/20...,"['89_0_홍대_2룸.jpg', '89_1_홍대_2룸.jpg', '89_2_홍대_...",빌라,서울,NaN,"50,000",50,2,7,0,홍대에 있는 방 2개 주택입니다.
1848,1831,1,경리단길 윤싸롱 3호점 - 채광 굿 깔끔하고 모던한 자취방,['https://s3.hourplace.co.kr/image/user/6876/2...,"['1_0_경리단길_윤싸롱_3호점_-_채광_굿_깔끔하고_모던한_자취방.jpg', '...",원룸,서울,NaN,"40,000",33,1,3,1,낮에 채광 잘드는 1층 집이고 공간은 약 10평정도 됩니다. 드라마 이태원클라쓰 촬...
1849,1832,2,"상수역 6분거리 , 합정역 6분거리 30평급 넓은 공간 자연광 스튜디오 VILLA ...",['https://s3.hourplace.co.kr/image/user/6691/2...,"['2_0_상수역_6분거리_,_합정역_6분거리_30평급_넓은_공간_자연광_스튜디오_...",스튜디오,서울,NaN,"45,000",96,3,6,1,30평의 넓은 공간에 동남서 방향 3면에 창이 배치되어 자연광이 풍부하고 섹션별로 ...
1850,1840,10,시매,['https://s3.hourplace.co.kr/image/user/6818/2...,"['10_0_시매.jpg', '10_1_시매.jpg', '10_2_시매.jpg', ...",주택,서울,NaN,"41,000",46,4,3,2,해방촌 언저리에 있는 작은 옥탑방이예요.\r\n\r\n방문이 없는 투룸 과 주방 공...
1851,1847,17,(용산) 채광맛집 예술가의 로프트!,['https://s3.hourplace.co.kr/temp/images/2020/...,"['17_0_(용산)_채광맛집_예술가의_로프트!.jpg', '17_1_(용산)_채광...",원룸,서울,NaN,"87,000",26,12,3,2,안녕하세요!\r\n\r\n용산에 위치한 복층 오피스텔입니다\r\n공간이 넓지는 않지...


In [823]:
## description 다듬기
def getDescList(n):
    okt=Okt()
    descList=[]
    for i in range(n):
        tempList=[]
        descript=(dataset.description[i]) # 한글자 이상만 뽑음
        descript=descript.replace("\n","")
        descript=descript.replace("\r","")
        descript=descript.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣09]","")
        tempList=[j[0] for j in okt.pos(descript) if ((len(j[0])>1)& (len(j[0])<7)& (j[1]=="Noun"))]
#         tempList=list(set(tempList))
        if len(tempList)>0:
            descList.append(tempList)
        else:
            descList.append(tempList)
#             print(tempList,"|",i)
                                    # [] | 92
                                    # [] | 988
                                    # [] | 1486
                                    # [] | 1806
    return descList

In [821]:
# descList=getDescList(len(dataset))
# descList

[] | 92
[] | 988
[] | 1486
[] | 1806


In [824]:
len(descList)

1852

In [825]:
## Tag 다듬기
## 복합명사로 이루어진 태그는 잘라서 길이 2~7 사이의 단어만 추출
def getTagList(n):
    parseTagList=[]
    okt=Okt()
    for i in range(n):
        tempTagList=[]
        tags=str(dataset.tag[i]).split('#') # 태그별로 자르기
        for tag in tags:
            if len(tag)>1:
                okt.pos(tag)  # 1개 태그 내 복합 명사 자르기
                for j in okt.nouns(tag):
                    if(len(j)>=2|len(j)<7):
                        tempTagList.append(j)
        tempTagList=list(set(tempTagList))
        parseTagList.append(tempTagList)
    return parseTagList

In [678]:
# tagList=getTagList(10)
# tagList

### CoreTag 2가지 방법

In [826]:
## Case 1) 코어 태그 추출 (태그, description 합집합)
def getCoreTags1(n):
    coreTagList1=[]
    okt=Okt()    
    coreTag=[]
    descript=getDescList(n) # 한글자 이상만 뽑음
    tags=getTagList(n)  # tag를 한번 더 파싱
    for i in range(len(descript)): 
        if (len(tags[i])>0): 
            coreTag.append(tags[i]+descript[i])
        else:
            coreTag.append(descript[i])
    coreTagList1.append(coreTag)
    return coreTagList1

In [827]:
%time CoreTagDate1=getCoreTags1(len(dataset))
CoreTagDate1=CoreTagDate1[0]
# CoreTagDate1

Wall time: 2min 6s


In [828]:
## Case 2) 코어 태그 추출 (부분적으로 기존 태그 + description + coreTag )
def getCoreTags2(n):
    coreTagList=[]
    okt=Okt()    
    descript=getDescList(n) # 한글자 이상만 뽑음
    tags=getTagList(n)  # tag를 한번 더 파싱
    for k in range(len(descript)):
        coreTags=[]
        for t in range(len(tags[k])):
            if (len(tags[k][t])>0)&(tags[k][t] in descript[k]): # 태그가 description 단어에도 있는지 확인
                coreTags.append(tags[k][t])
        if (len(tags[k])<=3): # 태그 수가 없으면 디스크립션으로
            coreTagList.append(descript[k])
        elif((len(tags[k])>3)&(len(coreTags)<3)):  # 디스크립션과 일치하는 태그가 없는데 기존 태그 많으면 기존 태그 사용
            coreTagList.append(tags[k])
        elif(len(coreTags)>=3): # 디스크립션과 일치하는 태그가 많으면 일치태그 사용
            coreTagList.append(coreTags)
    return coreTagList

In [829]:
%time CoreTagDate2=getCoreTags2(1852)
# CoreTagDate2

Wall time: 2min 6s


In [830]:
print(len(CoreTagDate1),len(CoreTagDate2))

1852 1852


## wordcloud
    단어간 유사도

### CoreTag로 유사 키워드 추출

In [ ]:
import gensim 
from gensim.models import word2vec


In [838]:
num_features = 100    # Word vector dimensionality                      
min_word_count = 4   # Minimum word count                        
num_workers = 2     # Number of threads to run in parallel
context = 4          # Context window size                                                                                    
downsampling = 1e-3  # Downsample setting for frequent words

model1 = gensim.models.Word2Vec(CoreTagDate1, workers=num_workers, 
                          size=num_features, min_count = min_word_count,
                          window = context, sample = downsampling)

model2 = gensim.models.Word2Vec(CoreTagDate2, workers=num_workers, 
                          size=num_features, min_count = min_word_count,
                          window = context, sample = downsampling)

model_name = "TagSimilarity1"
model1.save(model_name)

model_name = "TagSimilarity2"
model2.save(model_name)

In [839]:
# model.wv.vocab
test_keyword=["가정","사진","촬영","원목","아파트","루프","자연","채광","스튜디오","광고","거실","주택","파티","빌라","카페",
              "빈티","플라워","홍대","공간","화이트","느낌","분위기","원룸",
              "사무실","서울","마당","정원","모던","인테리어","오피스"]

testkeywordDict1={}
for keyword in test_keyword:
    testkeywordDict1.update({keyword:model1.wv.most_similar(keyword,topn=5)})
testkeywordDict1

{'가정': [('아파트', 0.9997988343238831),
  ('화이트', 0.9996860027313232),
  ('일반', 0.9996804594993591),
  ('신혼집', 0.9996777772903442),
  ('거실', 0.9996752738952637)],
 '사진': [('장소', 0.9998262524604797),
  ('활용', 0.9997926354408264),
  ('배경', 0.9997895956039429),
  ('소품', 0.9997485876083374),
  ('진행', 0.999746561050415)],
 '촬영': [('영상', 0.9984505772590637),
  ('장소', 0.9982472062110901),
  ('제품', 0.9982207417488098),
  ('쇼핑몰', 0.9981820583343506),
  ('광고', 0.9981671571731567)],
 '원목': [('우드', 0.9998226761817932),
  ('모던', 0.9997041821479797),
  ('미니', 0.9997035264968872),
  ('하우스', 0.9996861815452576),
  ('주방', 0.9996737837791443)],
 '아파트': [('가정', 0.9997988343238831),
  ('남향', 0.9996756315231323),
  ('거실', 0.9996610283851624),
  ('신혼집', 0.9996519684791565),
  ('일반', 0.9996455907821655)],
 '루프': [('옥상', 0.999893069267273),
  ('서울', 0.9998436570167542),
  ('야외', 0.9998264312744141),
  ('단독주택', 0.9998171925544739),
  ('정원', 0.9998037815093994)],
 '자연': [('감성', 0.9998098611831665),
  ('스튜디오', 0.99

In [840]:
# "편안" 키워드 반영 X
test_keyword=["가정","사진","촬영","원목","아파트","루프","자연","채광","스튜디오","광고","거실","주택","파티","빌라","카페",
              "빈티","플라워","홍대","공간","화이트","느낌","분위기","원룸",
              "사무실","서울","마당","정원","모던","인테리어","오피스"]
testkeywordDict2={}
for keyword in test_keyword:
    testkeywordDict2.update({keyword:model2.wv.most_similar(keyword,topn=5)})
testkeywordDict2

{'가정': [('촬영', 0.9990453124046326),
  ('공간', 0.9989565014839172),
  ('위치', 0.9989500641822815),
  ('주차', 0.9989433288574219),
  ('사용', 0.9989326596260071)],
 '사진': [('촬영', 0.9995864629745483),
  ('위치', 0.9995071887969971),
  ('공간', 0.9994994401931763),
  ('아파트', 0.999497652053833),
  ('시간', 0.9994531869888306)],
 '촬영': [('위치', 0.9997480511665344),
  ('공간', 0.9997295141220093),
  ('시간', 0.9997019171714783),
  ('느낌', 0.9996501207351685),
  ('주차', 0.9996204972267151)],
 '원목': [('공간', 0.9988971948623657),
  ('거실', 0.9988398551940918),
  ('위치', 0.9988173246383667),
  ('완비', 0.9987798929214478),
  ('화이트', 0.9987741708755493)],
 '아파트': [('공간', 0.9996215105056763),
  ('촬영', 0.9995707869529724),
  ('위치', 0.999561071395874),
  ('인테리어', 0.9995355606079102),
  ('시간', 0.9995321035385132)],
 '루프': [('촬영', 0.9991289377212524),
  ('공간', 0.9991270899772644),
  ('위치', 0.9991092085838318),
  ('분위기', 0.998996376991272),
  ('대관', 0.9989961981773376)],
 '자연': [('촬영', 0.9995615482330322),
  ('공간', 0.99951100

## 각 모델 결과 태그를 새로운 태그 (extend_tag1,extend_tag) 컬럼으로 저장

In [849]:
extendTagList=[]
vocab=model2.wv.vocab
tagList1=[]
for i in range(len(CoreTagDate1)):
    extendTag=[]
    tag1=[]
    try:
        for j in range (len(CoreTagDate1[i])):
            if(CoreTagDate1[i][j] in vocab.keys()):
#                 print(CoreTagDate1[i][j])
#                 extendSet=model1.wv.most_similar(CoreTagDate1[i][j],topn=2)
                tag1.append(CoreTagDate1[i][j])
#                 for extend in extendSet:
#                     extendTag.append(extend)
#                 extendTagList.append(list(set(extendTag[0])))
            else:
                continue
        tagList1.append(tag1)
    except KeyError:
        continue

In [852]:
# dataset["new_tag1"]=tagList1

0       [서울시, 모던, 분리, 원룸, 서교동, 빌라, 홍대, 클래식, 홍대, 메인, 거리...
1       [전망, 벽난로, 루프, 자연, 고풍, 내부, 거실, 키친, 음악, 작업, 스튜디오...
2       [전망, 벽난로, 루프, 자연, 고풍, 내부, 거실, 키친, 음악, 작업, 스튜디오...
3       [신축, 아파트, 가정, 위치, 신축, 투룸, 오피스텔, 남향, 느낌, 야경, 신축...
4       [분위기, 쇼파, 가구, 부부, 공간, 신축, 거실, 거실, 서재, 주방, 공간, ...
                              ...                        
1847                                             [홍대, 주택]
1848       [채광, 공간, 평정, 드라마, 이태원, 촬영, 바로, 동네, 자연, 상품, 촬영]
1849    [평의, 공간, 방향, 배치, 자연, 디자인, 공간, 촬영, 제품, 촬영, 영상, ...
1850    [해방촌, 옥탑방, 방문, 투룸, 주방, 공간, 가지, 공간, 보시, 우드, 공간,...
1851    [용산, 위치, 복층, 오피스텔, 공간, 고가, 미터, 느낌, 남향, 통창, 정도,...
Name: new_tag1, Length: 1852, dtype: object

In [851]:
# extendTagList=[]
vocab=model2.wv.vocab
tagList2=[]
for i in range(len(CoreTagDate2)):
    extendTag=[]
    tag2=[]
    try:
        for j in range (len(CoreTagDate2[i])):
            if(CoreTagDate2[i][j] in vocab.keys()):
#                 print(CoreTagDate2[i][j])
#                 extendSet=model2.wv.most_similar(CoreTagDate2[i][j],topn=2)
                tag2.append(CoreTagDate2[i][j])
#                 for extend in extendSet:
#                     extendTag.append(extend)
#                 extendTagList.append(list(set(extendTag[0])))
            else:
                continue
        tagList2.append(tag2)
    except KeyError:
        continue

In [853]:
dataset["new_tag2"]=tagList2

In [860]:
dataset.to_csv('C:/바탕 화면/playdata/ice/eclipse/Devils_Project/dataWork/controller/total_dataset_trim_newTag.csv',
                 sep=',',na_rep='NaN',index =True,encoding='utf-8-sig') # do not write index


In [857]:
dataset[1830:1840]

,NO,img_fold_no,studio_name,studio_picture,imafe_file,category,region,tag,price,area,floor,default_person,parking,description,new_tag1,new_tag2
1830,1528,129,서울 남산 소월길 힐튼뷰 테라스 자연 채광 렌탈 스튜디오,['https://s3.hourplace.co.kr/image/user/7276/2...,"['129_0_서울_남산_소월길_힐튼뷰_테라스_자연_채광_렌탈_스튜디오.jpg', ...",스튜디오,서울,NaN,"44,000",132,3,4,4,안녕하세요\r\n\r\n후암동_남산 소월길\r\n서울 중심에 위치한 120평 규모의...,"[후암동, 남산, 서울, 중심, 위치, 규모, 자연, 채광, 렌탈, 스튜디오, 저희...","[후암동, 남산, 서울, 중심, 위치, 규모, 자연, 채광, 렌탈, 스튜디오, 저희..."
1831,1538,139,"상수역 6분거리 , 합정역 6분거리 30평급 넓은 공간 자연광 스튜디오 VILLA ...",['https://s3.hourplace.co.kr/image/user/6691/2...,"['139_0_상수역_6분거리_,_합정역_6분거리_30평급_넓은_공간_자연광_스튜디...",스튜디오,서울,NaN,"45,000",96,3,6,1,30평의 넓은 공간에 동남서 방향 3면에 창이 배치되어 자연광이 풍부하고 섹션별로 ...,"[평의, 공간, 방향, 배치, 자연, 디자인, 공간, 촬영, 제품, 촬영, 영상, ...","[평의, 공간, 방향, 배치, 자연, 디자인, 공간, 촬영, 제품, 촬영, 영상, ..."
1832,1591,192,홍대망원연습실 -중세시대 연습실-,['https://s3.hourplace.co.kr/temp/images/2019/...,"['192_0_홍대망원연습실_-중세시대_연습실-.jpg', '192_1_홍대망원연습...",스튜디오,서울,NaN,"12,000",17,B1,5,0,"※SBS 더팬 ,KBS 댄싱하이,MBC언더나인틴,M.net 2019쇼미더머니 촬영-...","[촬영, 시설, 홍대, 학생, 복합, 예술, 공간, 연습실, 최상, 인테리어, 엔틱...","[촬영, 시설, 홍대, 학생, 복합, 예술, 공간, 연습실, 최상, 인테리어, 엔틱..."
1833,1608,209,홍대망원 보컬전문녹음실 ASMR,['https://s3.hourplace.co.kr/temp/images/2019/...,"['209_0_홍대망원_보컬전문녹음실_ASMR.jpg', '209_1_홍대망원_보컬...",스튜디오,서울,NaN,"12,000",7,B1,5,0,※24시 운영 노래녹음 및 음반작업-각종사운드 녹음실■ASMR■\r\n\r\n■NE...,"[운영, 노래, 녹음, 작업, 각종, 사운드, 녹음실, 녹음, 장비, 컴퓨터, 시설...","[운영, 노래, 녹음, 작업, 각종, 사운드, 녹음실, 녹음, 장비, 컴퓨터, 시설..."
1834,1652,253,홍대 자연채광 좋은 디알스튜디오 B룸,['https://s3.hourplace.co.kr/image/user/7393/2...,"['253_0_홍대_자연채광_좋은_디알스튜디오_B룸.jpg', '253_1_홍대_자...",스튜디오,서울,NaN,"60,000",53,5,3,1,6월 신규오픈한 디알스튜디오 B룸입니다.\r\n정남향 3면 채광으로 주간 내내 밝습...,"[오픈, 스튜디오, 남향, 채광, 창가, 침대, 설치, 다른, 느낌, 구성]","[오픈, 스튜디오, 남향, 채광, 창가, 침대, 설치, 다른, 느낌, 구성]"
1835,1659,260,마곡 마음사진관,['https://s3.hourplace.co.kr/temp/images/2019/...,"['260_0_마곡_마음사진관.jpg', '260_1_마곡_마음사진관.jpg', '...",스튜디오,서울,NaN,"44,000",99,8,1,50,깔끔하고 채광좋은 예쁜 스튜디오.\r\n주차가능,"[채광, 스튜디오, 주차, 가능]","[채광, 스튜디오, 주차, 가능]"
1836,1681,282,"[플래티늄]이국적 컨셉의 루프탑, 자연광 스튜디오",['https://s3.hourplace.co.kr/image/user/7910/2...,"['282_0_[플래티늄]이국적_컨셉의_루프탑,_자연광_스튜디오.jpg', '282...",스튜디오,서울,NaN,"20,000",116,5,3,1,RENT\r\n___\r\n\r\n자연광과 함께 다양한 배경과 소품을 통해 여러 연...,"[자연, 배경, 소품, 통해, 여러, 연출, 호리, 루프, 렌탈, 서비스, 모델, ...","[자연, 배경, 소품, 통해, 여러, 연출, 호리, 루프, 렌탈, 서비스, 모델, ..."
1837,1718,3,별내 복층 다가구 주택,['https://s3.hourplace.co.kr/temp/images/2019/...,"['3_0_별내_복층_다가구_주택.jpg', '3_1_별내_복층_다가구_주택.jpg...",빌라,경기,NaN,"87,000",264,1,10,4,1층은 모던하고 시크한 느낌이고 2층은 1층과는 정반대의 따뜻한 느낌의 공간입니다,"[모던, 느낌, 느낌, 공간]","[모던, 느낌, 느낌, 공간]"
1838,1720,5,주방 과 거실,['https://s3.hourplace.co.kr/temp/images/2019/...,"['5_0_주방_과_거실.jpg', '5_1_주방_과_거실.jpg']",빌라,충남,NaN,"62,000",76,1,7,3,주방과 이어진 거실,"[주방, 거실]","[주방, 거실]"
1839,1730,15,삼성동 진흥빌라 3차,['https://s3.hourplace.co.kr/temp/images/2019/...,['15_0_삼성동_진흥빌라_3차.jpg'],빌라,서울,NaN,"186,000",231,2,5,2,오래된 빨강벽돌 빌라 입니다.\r\n10세대이고 빨강벽난로가 있는 거실이 있습니다....,"[벽돌, 빌라, 세대, 벽난로, 거실, 나무, 바닥, 분위기, 가정, 조금]","[벽돌, 빌라, 세대, 벽난로, 거실, 나무, 바닥, 분위기, 가정, 조금]"
